In [2]:
import operator
import numpy as np
import math
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import matplotlib.animation as animation

# %matplotlib nbagg

class SimulatedAnnealing():
    def __init__(self, minimize, T_min, T_max, init,
                 neighbour, energy, accept, cooling):
        self.compare = operator.lt if minimize else operator.gt
        self.T_min = T_min
        self.neighbour = neighbour
        self.energy = energy
        self.accept = accept
        self.cooling = cooling
        
        self.T = T_max
        self.best = init()
        self.step = 0

    def iteration(self):
        next = self.neighbour(self.T, self.best)
        deltaE = self.energy(next) - self.energy(self.best)
        if self.compare(deltaE, 0):
            self.best = next
        elif random.random() < self.accept(self.T, deltaE):
            self.best = next

    def cool(self):
        self.T = self.cooling(self.T, self.best)

    def execute(self):
        while self.T > self.T_min:
            self.iteration()
            self.cool()
            self.log()
            self.step += 1
        print('\nf(' + str(self.best) + ') = ' + str(self.energy(self.best)))
        return self.best
    
    def log(self):
        if(self.step % 100 == 0):
            print(str(self.step) + " T="+str(self.T) +
                  "\tf(" + str(self.best) + ") = " + str(self.energy(self.best)))

# standart configuration parameters and functions for Simulated Annealing
t_min = 1e-5
t_max = 1

def init():
    return np.array([0.1, 1.4])

def neighbour(T, x):
    #return x + random.uniform(-T, T, x.shape)
    return random.normal(x, max(T, 0.1), x.shape)

def accept(T, deltaE, k=0.1):
    return math.exp(-(deltaE)/k/T)

def cooling(T, best):
    return T*0.99

In [3]:
# plot the Rosenbrock function

def rosenbrock(x, y, a=1, b=100):
    return (a - x)**2 + b*(y - x**2)**2

def rosenbrock_energy(vector):
    return rosenbrock(*vector)

# Design variables at mesh points
i1 = np.arange(-2, 2, 0.01)
i2 = np.arange(-1, 3, 0.01)
X, Y = np.meshgrid(i1, i2)
Z = rosenbrock(X, Y)

fig = plt.figure()
ax = plt.axes(projection='3d')

Gx, Gy = np.gradient(Z) # gradients with respect to x and y
G = (Gx**2+Gy**2)**.5  # gradient magnitude
N = G/G.max()  # normalize 0..1

ax.plot_surface(X, Y, Z,facecolors=cm.jet(N))

plt.show()

In [4]:
# minimize the Rosenbrock function with Simulated Annealing

rosenbrock_min = SimulatedAnnealing(True, t_min, t_max, init,
                                    neighbour, rosenbrock_energy, accept, cooling).execute()

0 T=0.99	f([-0.82760275  1.60474883]) = 87.9474775475
100 T=0.36237201786049694	f([ 0.95040625  0.93405811]) = 0.0972376993656
200 T=0.13263987810938213	f([ 1.10789084  1.21211976]) = 0.0350566133491
300 T=0.0485504851305729	f([ 1.10858089  1.23509027]) = 0.0155581401459
400 T=0.017771047742294682	f([ 1.10825959  1.22998309]) = 0.0120242157863
500 T=0.006504778211990459	f([ 1.08543155  1.17550222]) = 0.00800580815753
600 T=0.0023809591983979563	f([ 1.02495244  1.05611609]) = 0.00374584840577
700 T=0.0008715080698656353	f([ 1.02495244  1.05611609]) = 0.00374584840577
800 T=0.00031900013925143135	f([ 1.02495244  1.05611609]) = 0.00374584840577
900 T=0.00011676436783668758	f([ 1.02879759  1.05410345]) = 0.00269642975104
1000 T=4.2739534936551264e-05	f([ 1.02879759  1.05410345]) = 0.00269642975104
1100 T=1.564405203775484e-05	f([ 1.02879759  1.05410345]) = 0.00269642975104

f([ 1.02879759  1.05410345]) = 0.00269642975104


In [5]:
# maximization of the 0-1 Knapsack Problem with data taken from
# https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html
# optimal profit = 1458
# optimal selection = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

DIM = 15
W = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
V = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
C = 750

def kp_init():
    return np.random.randint(2, size=DIM)

def kp_neighbour(T, knapsack):
    new_knapsack = np.copy(knapsack)
    new_knapsack[random.randint(DIM)] ^= 1
    while kp_cost(new_knapsack) == 0:
        new_knapsack[random.randint(DIM)] ^= 1
    return new_knapsack

def kp_cost(knapsack):
    total_weight = sum([W[i] for i, chosen in enumerate(knapsack) if chosen])
    total_value = sum([V[i] for i, chosen in enumerate(knapsack) if chosen])
    return 0 if total_weight > C else total_value

def kp_accept(T, deltaE, k=0.1):
    return math.exp(deltaE / k / T)

kp_max = SimulatedAnnealing(False, t_min, t_max, kp_init,
                            kp_neighbour, kp_cost, kp_accept, cooling).execute()

0 T=0.99	f([1 1 1 1 1 0 0 0 0 0 1 1 1 0 0]) = 1374
100 T=0.36237201786049694	f([0 1 1 0 1 1 1 0 0 1 0 0 1 0 1]) = 1442
200 T=0.13263987810938213	f([1 0 0 1 1 0 1 1 1 0 0 0 1 0 1]) = 1451
300 T=0.0485504851305729	f([1 0 0 1 1 0 1 1 1 0 0 0 1 0 1]) = 1451
400 T=0.017771047742294682	f([1 0 0 1 1 0 1 1 1 0 0 0 1 0 1]) = 1451
500 T=0.006504778211990459	f([1 0 0 1 1 0 1 1 1 0 0 0 1 0 1]) = 1451
600 T=0.0023809591983979563	f([1 0 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1452
700 T=0.0008715080698656353	f([1 0 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1452
800 T=0.00031900013925143135	f([0 1 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1456
900 T=0.00011676436783668758	f([0 1 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1456
1000 T=4.2739534936551264e-05	f([0 1 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1456
1100 T=1.564405203775484e-05	f([0 1 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1456

f([0 1 1 1 0 0 1 1 1 0 0 0 0 1 1]) = 1456


In [6]:
# animation of Rosenbrock minimization

sa = SimulatedAnnealing(True, t_min, t_max, init, neighbour, rosenbrock_energy, accept, cooling)
fig = plt.figure()
xmin, xmax = -2.0, 2.0
ymin, ymax = -1.0, 3.0
x1 = np.linspace(xmin, xmax, 100)
x2 = np.linspace(ymin, ymax, 100)
Xc, Yc = np.meshgrid(x1, x2)
Zc = rosenbrock(Xc, Yc)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False, xlim=(xmin, xmax), ylim=(ymin, ymax))
CS = ax.contour(Xc, Yc, Zc, levels=[0, 0.1, 1, 2, 10, 20, 100])
ax.clabel(CS, inline=1, fmt='%1.1f', fontsize=14)
ax.set_title('Non-Convex Function')
line, = ax.plot([], [], 'o-', lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
energy_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)

def init_animation():
    line.set_data([], [])
    time_text.set_text('')
    energy_text.set_text('')
    return line, time_text, energy_text

xs = []
ys = []
def animate(step):
    global sa, xs, ys
    sa.iteration()
    sa.cool()
    xs.append(sa.best[0])
    ys.append(sa.best[1])
    line.set_data(xs, ys)
    time_text.set_text('time = %.1f' % step)
    energy_text.set_text('temperature = %.3f J' % sa.T)
    return line, time_text, energy_text

ani = animation.FuncAnimation(fig, animate, frames=25, interval=25, init_func=init_animation)
plt.show()

In [7]:
def spike_pit(x, y):
    return 21.5 + x*np.sin(4*math.pi*x) + y*np.sin(20*math.pi*y)

xs = np.arange(-3, 12.1, 0.01)
ys = np.arange(4.1, 5.8, 0.01)
# xs = np.arange(10, 40, 0.01)
# ys = np.arange(-10, 10, 0.01)

X, Y = np.meshgrid(xs, ys)
Z = spike_pit(X, Y)

fig = plt.figure()
ax = plt.axes(projection='3d')

Gx, Gy = np.gradient(Z) # gradients with respect to x and y
G = (Gx**2+Gy**2)**.5  # gradient magnitude
N = G/G.max()  # normalize 0..1

ax.plot_surface(X, Y, Z,facecolors=cm.jet(N))
# ax.plot_surface(X, Y, Z)

plt.show()

In [8]:
# minimize the spike pit

def spike_init():
    return np.array([30.0, 0.0])

def spike_energy(vector):
    return spike_pit(*vector)

spike_pit_min = SimulatedAnnealing(False, t_min, t_max, spike_init,
                                   neighbour, spike_energy, kp_accept, cooling).execute()

0 T=0.99	f([ 29.0862361   -1.54151859]) = 47.9861843173
100 T=0.36237201786049694	f([ 31.1046855   -3.12015213]) = 54.5731892993
200 T=0.13263987810938213	f([ 31.12113287  -3.82620383]) = 56.3996568772
300 T=0.0485504851305729	f([ 31.12113287  -3.82620383]) = 56.3996568772
400 T=0.017771047742294682	f([ 31.12113287  -3.82620383]) = 56.3996568772
500 T=0.006504778211990459	f([ 31.12113287  -3.82620383]) = 56.3996568772
600 T=0.0023809591983979563	f([ 31.12113287  -3.82620383]) = 56.3996568772
700 T=0.0008715080698656353	f([ 31.12113287  -3.82620383]) = 56.3996568772
800 T=0.00031900013925143135	f([ 31.12113287  -3.82620383]) = 56.3996568772
900 T=0.00011676436783668758	f([ 31.12113287  -3.82620383]) = 56.3996568772
1000 T=4.2739534936551264e-05	f([ 31.12386294  -4.02494227]) = 56.6456015436
1100 T=1.564405203775484e-05	f([ 31.12386294  -4.02494227]) = 56.6456015436

f([ 31.12386294  -4.02494227]) = 56.6456015436


In [9]:
# animation of Spike Pit maximization

sa = SimulatedAnnealing(False, t_min, t_max, spike_init,
                                   neighbour, spike_energy, kp_accept, cooling)
fig = plt.figure()
xmin, xmax = 10, 40
ymin, ymax = -10, 10
x1 = np.linspace(xmin, xmax, 100)
x2 = np.linspace(ymin, ymax, 100)
Xc, Yc = np.meshgrid(x1, x2)
Zc = spike_pit(Xc, Yc)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False, xlim=(xmin, xmax), ylim=(ymin, ymax))
CS = ax.contour(Xc, Yc, Zc, levels=[20, 30, 40, 50, 60])
ax.clabel(CS, inline=1, fmt='%1.1f', fontsize=14)
ax.set_title('Trigonometric Spikes')
line, = ax.plot([], [], 'o-', lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
energy_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)

def init_animation():
    line.set_data([], [])
    time_text.set_text('')
    energy_text.set_text('')
    return line, time_text, energy_text

xs = []
ys = []
def animate(step):
    global sa, xs, ys
    sa.iteration()
    sa.cool()
    sa.log()
    sa.step += 1
    xs.append(sa.best[0])
    ys.append(sa.best[1])
    line.set_data(xs, ys)
    time_text.set_text('time = %.1f' % step)
    energy_text.set_text('temperature = %.3f J' % sa.T)
    return line, time_text, energy_text

ani = animation.FuncAnimation(fig, animate, frames=25, interval=25, init_func=init_animation)
plt.show()

0 T=0.99	f([ 30.   0.]) = 21.5
